In [1]:
# %pip install domolibrary --upgrade
# %pip install domolibrary_execution --upgrade

## WARNING
-- as of 2/12/2025 - the test_full_auth workflow does not support instances MFA


## SETUP
script should read from a list of configuration objects (account name, target instance, target user) and (re)generate an access token for that account if necessary

- generate local (config) auth
- Set the local / current user (needed for reading local accounts)
- generate local_user (dmdu.DomoUser)
- get local workspace (dmdj.DomoJupyterWorkspace) needed for reading account creds shared with this workspace



In [1]:
import domojupyter as dj
import domolibrary_execution.utils.domojupyter as dxdj
import domolibrary.client.DomoAuth as dmda

local_auth = await dxdj.generate_domo_auth('domo-community', domojupyter_fn = dj)

# await config_auth.who_am_i()

🎉 full_auth token retrieved from domo-community ⚙️


In [2]:
import domolibrary.classes.DomoUser as dmdu

CURRENT_USER = 'jae@datacrew.space'

local_user = await dmdu.DomoUsers.by_email(email_ls = [CURRENT_USER], auth = local_auth)

local_user

DomoUser(id='1893952720', display_name='Jae Wilson1', email_address='jae@datacrew.space', role_id=810756122, department='Business Improvement', title=None, avatar_key='/api/content/v1/avatar/USER/1893952720', phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, created_dt=datetime.datetime(2020, 5, 8, 17, 55, 18), last_activity_dt=datetime.datetime(2025, 2, 17, 16, 44, 19, 197000), custom_attributes={}, role=None, domo_api_clients=None, domo_access_tokens=None)

In [8]:
import domolibrary.classes.DomoJupyter as dmdj

WORKSPACE_ID = '70afbd1d-e85c-4846-8a0e-a46987011676'

local_workspace = await dmdj.DomoJupyterWorkspace.get_by_id(WORKSPACE_ID, auth = local_auth)  

## set up test

In [3]:
test_deploy = [
    {
        "target_email" : "automation@datacrew.space",
        "target_role" : "super_admin",
        "target_instance" : "datacrew-space",
        "target_account_name" : "sdk_automation",
        "default_share_email_ls" : ['anfavis@gmail.com'],
        "local_account_name" : "sdk_datacrew_automation"
    }
]

test_config = test_deploy[0]


## Route Functions 

### route functions for Config Class


In [9]:
import domolibrary.classes.DomoAccount as dmac
import domolibrary.classes.DomoJupyter as dmdj

async def generate_backup_auth(target_instance):
    return await dxdj.generate_domo_auth(target_instance, domojupyter_fn = dj)

async def initial_deploy(local_account_name: str, 
                         target_instance : str,
                         local_auth : dmda.DomoAuth,
                         local_user : dmdu.DomoUser,
                         local_workspace: dmdj.DomoJupyterWorkspace,
                         debug_api: bool = False
                         ) -> dmdj.DomoJupyterAccount:
    
    """
    deploys an empty local account object according to config
    """
        
    #deploy local account
    local_account = await dmac.DomoAccounts.upsert_account(
        account_config = dmac.AccountConfig['domo_access_token'].value(),
        auth = local_auth,
        account_name = local_account_name,
        debug_api = debug_api
    )
    
    # retrieve local credential account
    local_account = await dmac.DomoAccount_Credential.get_by_id(
        auth = local_auth,
        account_id = local_account.id
    )
    
    # share local account with local user (so can read credentials)
    await local_account.Access.share(
                user_id=local_user.id,
                debug_api=debug_api,
            )
    
    local_account.target_instance =  target_instance
    
    
    return await local_workspace.add_account(domo_account = local_account,
                                             is_update_config = True,
                                             domo_user = local_user)
    

test_config['backup_auth'] = await generate_backup_auth(test_config['target_instance'])

test_dja = await initial_deploy(
    local_account_name = test_config['local_account_name'],
    target_instance = test_config['target_instance'],
    local_auth = local_auth,
    local_workspace = local_workspace,
    local_user = local_user
)

test_dja

🎉 full_auth token retrieved from datacrew-space ⚙️


DomoJupyterAccount(account_id=148, alias='sdk_datacrew_automation', is_exists=False, domo_account=DomoAccount_Credential(id=148, name='sdk_datacrew_automation', data_provider_type='domo-access-token', created_dt=datetime.datetime(2025, 2, 17, 17, 9, 15), modified_dt=datetime.datetime(2025, 2, 17, 17, 35, 51), owners=None, is_admin_summary=False, target_instance='datacrew-space', is_valid_full_auth=None, is_valid_token_auth=None, target_auth=None, target_user=None))

In [17]:
import domolibrary_extensions.utils.xkcd_password as xkcd
import domolibrary.client.DomoError as dmde
import domolibrary_execution.client.ExecutionError as dxde
import domolibrary.classes.DomoRole as dmdr
import domolibrary.classes.DomoUser as dmdu
import domolibrary_extensions.utils.xkcd_password as xkcd

async def upsert_target_user(target_auth : dmda.DomoAuth,
                             target_email :str ,
                             target_role : str) -> dmdu.DomoUser:
    
    domo_role = await dmdr.DomoRoles(auth= target_auth).search_by_name(target_role)
    return await dmdu.DomoUsers(auth = target_auth).upsert(
        email_address = target_email,
        role_id = domo_role.id)

async def fix_password(
    domo_account : dmac.DomoAccount_Credential,
    
    new_password : str = None,
    target_email : str = None,
    target_role : str = None,
    is_force_password_required: bool = False,

    backup_auth: dmda.DomoAuth = None,
    is_test_auth : bool = True,
    is_update_account: bool = True,
    
    debug_api: bool = False,
    debug_prn: bool = False,
    **kwargs,
):
    if is_test_auth or not domo_account.target_auth:
        try:
            await domo_account.test_auths(backup_auth = backup_auth)
        except dmde.DomoError as e:
            print(e)

    if domo_account.is_valid_full_auth:
        return f"No Action - {domo_account.name} username / password creds are valid"

    if is_force_password_required and not domo_account.Config.username:
        domo_account.set_username(target_email)
    
    user_email = domo_account.Config.username
    
    if is_force_password_required and not user_email:
        raise dxde.ExecutionError(
            f"no username for {domo_account.id} - {domo_account.name}"
        )
    
    if not domo_account.target_auth:
        await domo_account.test_auths()
        
    if not domo_account.target_user:
        try:
            await domo_account.get_target_user()
        except dmde.DomoError as e:
            domo_account.target_user = await upsert_target_user(
                target_auth = domo_account.target_auth,
                target_email = target_email,
                target_role = target_role
            )
                                        
    message = f"pasword reset - {domo_account.name} - {domo_account.target_user.display_name}"

    if debug_prn:
        print(message)
        
    new_password = new_password or xkcd.generate_domo_password(process_fn=xkcd.process_domo_password_fn, delimiter="-")

    await domo_account.update_target_user_password(
        new_password=new_password,
        is_test_auths=False,
        is_update_account=is_update_account,
        backup_auth=backup_auth,
        debug_api=debug_api,
    )
    
    return message

print(await fix_password(
    domo_account = test_dja.domo_account,
    new_password = None,
    is_force_password_required = True,
    target_email = test_config['target_email'],
    target_role = test_config['target_role'],
    backup_auth = test_config['backup_auth'],
    is_test_auth = True,
    is_update_account = True,
    
    debug_api = False
))

dxdj.read_domo_jupyter_account(test_dja.domo_account.name, domojupyter_fn = dj)

🎉 token_auth token retrieved from datacrew-space ⚙️
🎉 full_auth token retrieved from datacrew-space ⚙️
No Action - sdk_datacrew_automation username / password creds are valid


{'password': 'FEISTY-cable-stole2025!',
 'domoAccessToken': 'd90048a83a3adfd09a530c24db827d126f02fd0441d35172',
 'username': 'automation@datacrew.space'}

In [16]:
import asyncio
from typing import List
import domolibrary.classes.DomoAccount as dmac
import domolibrary.classes.DomoAccessToken as dmat

async def fix_token(
    domo_account: dmac.DomoAccount_Credential,
    
    token_name : str = None,
    is_force_token_required : bool = False,
    token_update_threshold: int = 7 ,
    domo_access_tokens : List[dmat.DomoAccessToken] = None,
    
    backup_auth: dmda.DomoAuth = None,
    is_test_auth : bool = True,
    is_update_account: bool = True,
    debug_api: bool = False,
    debug_prn: bool = False,
    
    **kwargs,
):
    if is_test_auth or not domo_account.target_auth:
        try:
            await domo_account.test_auths(backup_auth = backup_auth)
        except dmde.DomoError as e:
            print(e)
                     
    '''
    retrieve the domoAccessToken.
    cannot retrieve via target_user b/c not all access_token_accounts will have an email
    '''
    domo_access_tokens = domo_access_tokens or (
        await dmat.DomoAccessTokens(auth = domo_account.target_auth).get()
    )
    token_name = token_name or domo_account.name
    dat = next((dat for dat in domo_access_tokens if dat.name == token_name), None)
    
    
    # is valid and not expiring
    if domo_account.is_valid_token_auth and dat and dat.days_till_expiration > token_update_threshold:
        message = f'No Action - {dat.name} - valid token not expiring'
        if debug_prn:
            print(message)
        
        return message

    # dat (expiring or not valid)
    if dat:
        message = f'Regenerate - {dat.name} token - token expiring or invalid'
        
        if debug_prn:
            print(message)
            
        await dat.regenerate(debug_api = debug_api)
        return message
    
    # dat not required
    if not is_force_token_required and not domo_account.Config.domo_access_token:
        message = f"No Action {domo_account.name}- token_required is False"
        
        if debug_prn:
            print(message)
            
        return message 
    
    # no dat
    if not domo_account.target_user:
        await domo_account.get_target_user()
    
    message = f'Generate new access_token for {domo_account.name}'
    
    if debug_prn:
        print(message)
            
    await domo_account.reset_access_token(
        token_name = token_name,
        debug_api = debug_api,
        duration_in_days = 90
    )
    
    return message


print(await fix_token(
    domo_account = test_dja.domo_account,
    is_force_password_required = True,
    is_force_token_required = True,
    target_email = test_config['target_email'],
    target_role = test_config['target_role'],
    backup_auth = test_config['backup_auth'],
    is_test_auth = True,
    is_update_account = True,
    debug_api = False
))

dxdj.read_domo_jupyter_account(test_dja.domo_account.name, domojupyter_fn = dj)

testing token: sdk_datacrew_automation: 🛑  DAC_NoAccessToken 🛑 - function: 🛑 DomoAccount_Credential 🛑 148 || no access_token stored in account - sdk_datacrew_automation || error
🎉 full_auth token retrieved from datacrew-space ⚙️
Domo User 2081510391 - automation@datacrew.space - via dl 2025-02-17 does not have any access tokens
Generate new access_token for sdk_datacrew_automation


{'password': 'FEISTY-cable-stole2025!',
 'domoAccessToken': 'd90048a83a3adfd09a530c24db827d126f02fd0441d35172',
 'username': 'automation@datacrew.space'}

### Route functions for Remote Deploy Class

In [ ]:
import domolibrary.classes.DomoGroup as dmdg
import domolibrary.classes.DomoUser as dmdu

async def generate_domo_group(auth : dmda.DomoAuth,
                              group_name : str,
                              owner_user_email_ls: List[str] = None,
                              owner_group_name_ls : List[str] = None,
                              debug_api : bool = False
                             ):
    
    domo_group = await dmdg.DomoGroups(auth = auth).upsert(
        group_name = group_name,
        group_type = dmdg.GroupType_Enum.CLOSED.value,
        description = "generated for managing account access",
        debug_api = debug_api,
    )

    group_owners = []
    domo_users = []
    
    if owner_user_email_ls:
        domo_users = await dmdu.DomoUsers(auth = auth).search_by_email(owner_user_email_ls, 
                                                                       only_allow_one = False,
                                                                       debug_api = debug_api)
        group_owners += domo_users
        
    if owner_group_name_ls:
        group_owners += await dmdg.DomoGroups(auth = auth).search_by_name(
            group_name = owner_group_name_ls,
            only_allow_one = False,
            debug_api = debug_api
        )
        
    await domo_group.Membership.add_owners(group_owners, debug_api = debug_api)
    
    await domo_group.Membership.add_members(domo_users, debug_api  = debug_api)
    
    return domo_group
    

async def deploy_account_to_remote_instance(remote_auth : dmda.DomoAuth, 
                                            account_name : str, 
                                            account_config : dmac.AccountConfig,
                                            debug_api : bool = False,
                                            share_default_group : dmdg.DomoGroup = None,
                                            owner_group_name_ls : List[str] = None,
                                            owner_user_email_ls : List[str] = None
) :
    
    
    remote_account = await dmac.DomoAccounts.upsert_account(auth = remote_auth,
                                                            account_name = account_name,
                                                            account_config = account_config,
                                                            debug_api = debug_api,
                                                           )
    
    if share_default_group:
        await remote_account.Access.upsert_share(
            domo_group = share_default_group,
            debug_api = debug_api
        )
        
    return remote_account

## Config and Remote Deploy Class

In [ ]:
import domolibrary.classes.DomoJupyter as dmdj

from domolibrary_execution.client.ExecutionError import ExecutionError
import domolibrary.utils.chunk_execution as dmce
import domolibrary.classes.DomoAccount as dmac
import domolibrary.classes.DomoRole as dmdr

from typing import List, Callable
from dataclasses import dataclass, field
from functools import partial

@dataclass
class RemoteDeploy:
    remote_auth : dmda.DomoAuth
    
    backup_auth: dmda.DomoAuth = None
    
    account_name : str = None
    owner_group_name_ls : List[str] = None
    owner_user_email_ls : List[str] = None
    
    share_default_group : dmdg.DomoGroup = None
            
    async def generate_default_group(self, debug_api: bool = False ):
        self.share_default_group = await generate_domo_group(
            auth =self.remote_auth,
            group_name = self.account_name,
            owner_user_email_ls = self.owner_user_email_ls,
            owner_group_name_ls =self.owner_group_name_ls,
            debug_api = debug_api
        )
        
        return self.share_default_group
    
    def generate_task(self):
        return partial(deploy_account_to_remote_instance,
                       remote_auth = self.remote_auth,
                       account_name = self.account_name,
                       share_default_group=self.share_default_group,
                       owner_group_name_ls = self.owner_group_name_ls,
                       owner_user_email_ls =self.owner_user_email_ls
        )
    
    
@dataclass
class Config:
    domo_instance : str
    local_account_name: str
    account_email : str
    account_role : str
    owner_user_email_ls : List[str]
    
    dja : dmdj.DomoJupyterAccount
    
    _domojupyter_fn : Callable
    domo_account : dmac.DomoAccount_Credential = None

    new_password : str = field(repr = False, default = None)
    
    is_force_password_required : bool = False
    is_force_token_required : bool = False
    token_update_threshold : int = 7
    
    remote_deploy : List[RemoteDeploy] = None

    def __post_init__(self):
        self._init_dja()

    def _init_dja(self):
        self.dja.read_creds(domojupyter_fn = self._domojupyter_fn)
        self.dja.domo_account.target_instance = self.domo_instance

        self.domo_account = self.dja.domo_account
    
    def _init_password(self):
        self.new_password = self.new_password or xkcd.generate_domo_password(process_fn=xkcd.process_domo_password_fn, delimiter="-")
    
    
    async def generate_target_auth(self, domo_instance : str = None):
        domo_instance = domo_instance or self.domo_instance
        
        return self.domo_account._set_target_auth( 
            valid_backup_auth = await dxdj.generate_domo_auth(domo_instance,
                                                        domojupyter_fn = self._domojupyter_fn)
        )
    
    async def generate_target_user(self):
        await config.generate_target_auth()

        try:
            target_role = await dmdr.DomoRoles(auth = self.domo_account.target_auth).search_by_name(
                role_name = role_name,
                debug_api = debug_api
            )
            
        except dmdr.

        try:
            self.domo_account.set_username( self.account_email)
            await config.domo_account.get_target_user()

        except dmdu.SearchUser_NoResults as e:
            await dmdu.DomoUsers(auth = config.domo_account.target_auth).upsert(email_address =config.account_email)

                                                        
    async def fix_password(self, 
                     backup_auth : dmda.DomoAuth = None,
                     is_test_auth: bool = True,
                     is_update_account: bool = True,
                           
                     debug_api : bool = False,
                     debug_prn: bool = False,
                     **kwargs
    ):
        return await fix_password(
            domo_account = self.domo_account,
            new_password = self.new_password,
            is_force_password_required = self.is_force_password_required,
            
            backup_auth = backup_auth,
            is_test_auth = is_test_auth,
            is_update_account = is_update_account,

            debug_api = debug_api,
            debug_prn = debug_prn,
    )
    
    async def fix_token(self,
                        backup_auth : dmda.DomoAuth = None,
                        is_test_auth: bool = True,
                        is_update_account: bool = True,
                        
                        domo_access_tokens : List[dmat.DomoAccessToken] = None,
                       is_force_token_required : bool = False,
                        
                        debug_api : bool = False,
                        debug_prn: bool = False,
                        **kwargs
    ):
        return await fix_token(
            domo_account = self.domo_account,
            token_name = self.local_account_name,
            is_force_token_required = is_force_token_required if is_force_token_required is not None else self.is_force_token_required,
            token_update_threshold = self.token_update_threshold ,
            domo_access_tokens = domo_access_tokens,

            backup_auth = backup_auth,
            is_test_auth = is_test_auth,
            is_update_account = is_update_account,
            debug_api = debug_api,
            debug_prn = debug_prn,
        )
    
    
    @classmethod
    def _from_json(cls, obj, workspace_djas: List[dmdj.DomoJupyterAccount], 
                   domojupyter_fn: Callable,
                   remote_deploy : List[RemoteDeploy] = None
                  ):
        account_name = obj['local_account_name']
        
        try:
            dja = next((dja for dja in workspace_djas if dja.domo_account.name == account_name))
        
        except:
            raise 
            ExecutionError(
                f'{account_name} not found in list of workspace accounts.  has it been shared?'
            )
                                                                                                                                  
        return cls(**obj, dja = dja, _domojupyter_fn = domojupyter_fn, remote_deploy=remote_deploy)
    
    async def test_and_fix_domo_account(
        self,

        backup_auth : dmda.DomoAuth = None,
        is_test_auth: bool = True,
        is_update_account: bool = True,

        domo_access_tokens : List[dmat.DomoAccessToken] = None,

        debug_api : bool = False,
        debug_prn: bool = False,

    ):

        await self.domo_account.test_auths()

        repair_functions = [self.fix_password, self.fix_token]

        for fn in repair_functions:
            await fn(
                domo_account=self.domo_account,
                backup_auth=backup_auth,
                is_test_auth = False,
                
                is_force_password_required=self.is_force_password_required,
                is_force_token_required = self.is_force_token_required,

                domo_access_tokens = domo_access_tokens,
                debug_api = debug_api,
                debug_prn = debug_prn)

        await self.domo_account.update_config(debug_api = debug_api)
        
        await asyncio.sleep(5)
        return await self.domo_account.test_auths()
    
    async def deploy_to_remote_instances(self, debug_api : bool = False,):
        
        if not self.remote_deploy:
            return True
    
        function_ls = []
        
        for rd in self.remote_deploy:
            rd.account_name = rd.account_name or self.local_account_name
            await rd.generate_default_group()
            
            function_ls.append(rd.generate_task()  )
            
        return await dmce.gather_with_concurrency(*[
            fn(account_config = self.domo_account.Config, debug_api = debug_api )
            for fn in function_ls], n = 10)

## sample implementation

In [ ]:


test_remote_auth = await dxdj.generate_token_auth('datacrew-space', domojupyter_fn = dj)

config = Config._from_json(
    test_deploy[0],
    workspace_access_token_accounts,
    domojupyter_fn = dj,
    
    remote_deploy = [
        RemoteDeploy(remote_auth = test_remote_auth)]
    )

config

In [ ]:
import domolibrary.classes.DomoUser as dmdu

config.domo_account.set_username(config.account_email)

try:
    await config.generate_target_auth()
    await config.domo_account.get_target_user()

except dmdu.SearchUser_NoResults as e:
    await dmdu.DomoUsers(auth = config.domo_account.target_auth).upsert(email_address =config.account_email)

# set_target_user(config.account_email)

In [ ]:
await config.generate_target_auth()

await config.fix_token(debug_prn = True, is_force_token_required = True)

## Deploy Remote Instance


In [ ]:
await config.deploy_to_remote_instances()